<a href="https://colab.research.google.com/github/ordevoir/Python/blob/main/17_%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D1%84%D0%B0%D0%B9%D0%BB%D0%B0%D0%BC%D0%B8_%D0%B8_%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3%D0%B0%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Путь к интерпретатору

In [1]:
import sys
print(sys.executable)

c:\Users\werna\miniconda3\envs\marl\python.exe


# Путь к рабочему каталогу

In [3]:
import os

work_dir = os.getcwd()           # возвращает путь к рабочему каталогу (get current working directory)
print(work_dir)

c:\Users\werna\Documents\GitHub\Python


In [3]:
import os
work_dir = os.path.abspath("")
print(work_dir)

c:\Users\werna\Documents\GitHub\Python


# Путь к исполняемому файлу

## Для файлов `*.ipynb`

In [4]:
file_path = globals()['_dh'][0]     # путь к исполняемому файлу
print(str(file_path))

c:\Users\werna\Documents\GitHub\Python


## Для файлов `*.py`

In [ ]:
# from os.path import abspath, dirname
# path = abspath(__file__)      # возвращает полное имя текущего файла
# location = dirname(path)      # извлекает путь из полного имени файла

## Универсальная функция

В jupyter не определена переменная `__file__`, но определена функция `get_ipython()`, в отличие от файлов, запускаемых в режиме терминала. Это обстоятельство позволяет выяснить, запускается ли фрагмент кода в jupyter.

In [5]:
def is_running_in_jupyter():
    try:
        __file__
        return False
    except NameError:
        return True

is_running_in_jupyter()

True

Исходя из этого можно написать универсальную функцию, возвращающую путь к исполняемому файлу:

In [6]:
def get_abs_path():
    if is_running_in_jupyter():
        path = globals()['_dh'][0]
    else:
        from os.path import abspath
        path = abspath(__file__)
    return str(path)

get_abs_path()

'c:\\Users\\werna\\Documents\\GitHub\\Python'

> Пути к исполняемому файлу и рабочий каталог могут не совпадать. В общем случае рабочий каталог можно менять, в то время как расположение исполняемого файла будет прежним:

In [ ]:
import os

origin_wd = os.getcwd()                 # текущий рабочий каталог
os.chdir("..")                          # сменим рабочий каталог
file_path = get_abs_path()              # путь к исполняемому файлу
work_dir = os.getcwd()                  # путь к рабочему каталогу
os.chdir(origin_wd)                     # возврат к исходному каталогу
print(f"{file_path = }\n{work_dir =  }")

По умолчанию текущим рабочим каталогом будет каталог, из которого запускается файл в коммандной сторке, а не каталог, в котором расположен исполняемый файл.

Кроме того, пути поиска файлов не имеют никакого отношения к путям поиска модулей.

# Прочие функции


`os.mkdir('folder')` создает новую директорию

`os.makedirs('another1/another2/another3')` – сделать несколько вложенных папок.

`os.rename("text.txt", "renamed-text.txt")` – переименование файла или папки.

Переместить (с заменой) файл в другой каталог (переименовать полный путь):

`os.replace("renamed-text.txt", "folder/renamed-text.txt")`.

`os.path.isdir('folder')` возвращает `True`, если каталог `folder` существует.

`os.path.exists()` возвращает `True`, если файл/каталог с заданным именем существует.

`os.path.dirname()` отбрасывает имя файла и возвращает путь.

`os.path.basename()` отбрасывает путь и оставляет только имя файла/каталога. Например, `'foo/bar'` превратится в `'bar'`, а `'foo/bar/'` превратится в `''`.

`os.path.splitext()` позволяет отделить расширение от имени файла

In [8]:
from os.path import dirname, basename, splitext

path = "files/images/picture.png"
location = dirname(path)
filename, extension = splitext(basename(path))

location, filename, extension

('files/images', 'picture', '.png')

In [9]:
os.path.abspath(path)

'c:\\Users\\werna\\Documents\\GitHub\\Python\\files\\images\\picture.png'

In [12]:
os.path.abspath("")

'c:\\Users\\werna\\Documents\\GitHub\\Python'

In [11]:
basename(location)

'images'

# Чтение файла и запись

In [ ]:
# Чтение
with open('./file', 'r') as f:
    print(type(f))
    # f.read()         # получить содержимое файла
    # f.readline()     # построчное стение
    # f.readlines()    # получить список строк
    for line in f:   # перебор строк файла
        print(line)

# Запись
# with open('./file', 'w') as file:
#     file.write('some text')            # запись в файл
#     file.writelines(['some', 'text'])  # запись списка строк в файл

<class '_io.TextIOWrapper'>
some textsometext


Переменная `file` является объектому класса _io.TextIOWrapper. С другими методами этого класса можно ознакомиться <a href="https://www.w3schools.com/python/python_ref_file.asp">здесь</a>

# Скачивание файла

## `requests`

Один из сопособов – воспользоваться модулем `requests`. Функция `get()` возвращает объект класса `Response`. Поле `status_code` содержит статус ответа, а поле `content` – содержимое ответа в байтовом представлении (объект `bytes`).

Приведенная функция `download()` по умолчанию загружает файл в рабочий каталог, но также можно задать путь (абсолютный или относительный).

In [ ]:
import requests
from os import mkdir
from os.path import basename, isdir

def download(url, location='.'):
    response = requests.get(url)
    if response.status_code != 200:
        raise IOError(f"response statis code is {response.status_code}")

    if not isdir(location):
        mkdir(location)

    path = location + '/' + basename(url)
    with open(path, "wb") as file:
        file.write(response.content)

    print(f'The file is successfully downloaded with path: \n{path}')

In [ ]:
url = "https://raw.githubusercontent.com/ordevoir/Python/main/images/example_img.png"
download(url)

## `urlretrieve()`

Другой сопосб – использование функции `urlretrieve()` из модуля `urllib.request`. Функция `urlretrieve` копирует объект на диск. Первый аргумент – **url** объекта. Если второй аргумент (`filename`) не задан, то объект будет сохранен во временных файлах. Можно задать `filename`, и тогда объект будет сохранен в соответствующем месте с соответствующим именем. Функция `urlretrieve` возвращает кортеж `(filename, headers)` – имя файла и заголовки запроса.

Важное отличие следующей функции `download()` от предыдущей в том, что здесь файл не будет перезаписан, если он уже существует.

In [ ]:
from os.path import basename, exists
from urllib.request import urlretrieve

def download(url, location='.'):
    path = location + '/' + basename(url)
    if not exists(path):
        local_filename, headers = urlretrieve(url, path)
        print('Downloaded ' + path)
    else:
        print('file ' + path + ' already exists')

## Скачивание модуля из репозитория

Разместим файл модуля в той же директории, где и исполняемый файл:

In [ ]:
url = "https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/tools.py"

path = get_abs_path()
download(url, path)